In [2]:
from openai import OpenAI

client = OpenAI(
  api_key=""
)

assistant = client.beta.assistants.create(
  name="SBIR Assistant",
  instructions="You are an expert on government grants and funding. You are able to answer questions about the grants and funding available to small businesses.",
  model="gpt-4o-mini",
  tools=[{"type": "file_search"}],
)

In [3]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="SBIR Grants")

# Ready the files for upload to OpenAI
file_paths = ["./DoD_grants.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [4]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [13]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Use the PDF file Dod_grants that is in your vector store. Only use that to answer this prompt: Give me all grants related to supply chain vulnerabilities.",
    }
  ]
)

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

None


In [14]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

Here are the grants related to supply chain vulnerabilities identified in the document "Dod_grants":

1. **Resilience against Supply Chain Cyber Vulnerabilities** (N251-042)
   - **Objective**: Develop technology that ensures computing hardware technologies integrated into future combat systems are trustworthy and cyber secure.
   - **Description**: Focuses on ensuring the security, robustness, and overall performance of distributed shipboard computing infrastructure which includes over 3,000 CPU cores distributed across military-grade cabinets. The grant is aimed at improving the resilience of computational infrastructure amidst evolving cyber threats.
   
2. **Smart Contracts for Supply Chain Risk Management (SCRM)** (N251-068)
   - **Objective**: Develop an acquisition and sustainment contracting framework utilizing Smart Contracts for SCRM, employing blockchain technology.
   - **Description**: This initiative enhances visibility into supply chain processes and vendor activities, p